# Crawler
## 1. Download and persist #
- for a given URL `download()` method downloads binary data and stores in `self.content`. It returns `True` for success, else `False`.
- `persist()` method saves `self.content` somewhere in file system. We do it to avoid multiple downloads (for caching in other words).
- `load()` method loads data from hard drive. Returns `True` for success.

Tests checks that your code somehow works.

In [106]:
import requests
from urllib.parse import quote

class Document:
    
    def __init__(self, url):
        self.url = url

        
    def get(self):
        if not self.load():
            if not self.download():
                raise FileNotFoundError(self.url)
            else:
                self.persist()
    
    def download(self):
        r = requests.get(self.url, allow_redirects=True) # Getting contents
        if r: 
          self.content = r.content
          return True
        return False
    
    def persist(self):
        #TODO write document content to hard drive
        # url is attached with a file name for uniqueness

        url = quote(self.url, safe = "")
        open("nfile_" + url, 'wb').write(self.content)
            
    def load(self):
        try:
          url = quote(self.url, safe = "")
          f = open("nfile_" + url, "rb")
          self.content = f.read()
          return True
        except FileNotFoundError:
          return False
        return False

### 1.1. Tests ###

In [107]:
doc = Document('http://sprotasov.ru/data/iu.txt')

doc.get()
assert doc.content, "Document download failed"
assert "Code snippets, demos and labs for the course" in str(doc.content), "Document content error"

doc.get()
assert doc.load(), "Load should return true for saved document"
assert "Code snippets, demos and labs for the course" in str(doc.content), "Document load from disk error"

## 2. Parse HTML ##
`BeautifulSoap` library is a de facto standard to parse XML and HTML documents in python. Use it to complete `parse()` method that extracts document contents. initialization:
- `self.anchors` list of tuples `('text', 'url')` met in a document. Be aware, there exist relative links (e.g. `../content/pic.jpg`). Use `urllib.parse.urljoin()` to fix this issue.
- `self.images` list of images met in a document. Again, links can be relative to current page.
- `self.text` should keep plain text of the document without scripts, tags, comments and so on. You can refer to [this stackoverflow answer](https://stackoverflow.com/a/1983219) for details.

In [ ]:
from bs4 import BeautifulSoup, SoupStrainer
from bs4.element import Comment
import urllib.parse
from urllib.parse import urljoin
import re
import json


class HtmlDocument(Document):

    def parse(self):
        ancrs = []
        images = []

        soup = BeautifulSoup(self.content, 'html.parser')
        
        # Finding links with anchor tags in the data

        ancrs = soup.find_all('a')
        ancrs = [(anc.contents[0], urljoin(self.url, anc.get('href',''))) for anc in ancrs]
        
        # Finding images with image tags in the data

        imgs = soup.find_all('img')
        imgs = [urljoin(self.url, img.get('src','')) for img in imgs]
        
        # Finding the text from the data.

        txts = soup.find_all(text = True)
        text = ''.join(str(e) for e in txts)
        
        # print(len(imgs))
        self.anchors = ancrs
        self.images = imgs
        self.text = text

### 2.1. Tests ###

In [109]:
doc = HtmlDocument("http://sprotasov.ru")
doc.get()
doc.parse()

assert "тестирующий сервер codetest" in doc.text, "Error parsing text"
assert "http://sprotasov.ru/images/phone.png" in doc.images, "Error parsing images"
assert any(p[1] == "http://university.innopolis.ru/" for p in doc.anchors), "Error parsing links"

## 3. Document analysis ##
Implementing word and sentence splitting . `get_word_stats()` method returns `Counter` object.

In [114]:
from collections import Counter
import wordcounter


class HtmlDocumentTextData:
    
    def __init__(self, url):
        self.doc = HtmlDocument(url)
        self.doc.get()
        self.doc.parse()
    
    def get_sentences(self):
        # Spliting the text to get the sentences
        result = self.doc.text.splitlines()
        return result
    
    def get_word_stats(self):
        
        # First we got sentences
        sentences =  self.get_sentences()

        # Spliting the words

        wordfreq = []
        for sentence in sentences:
            wordfreq.extend(re.split(' +', sentence.lower()))
        
        return Counter(wordfreq)

### 3.1. Tests ###

In [115]:
doc = HtmlDocumentTextData("https://innopolis.university/")

print(doc.get_word_stats().most_common(10))
assert [x for x in doc.get_word_stats().most_common(10) if x[0] == 'иннополис'], 'иннополис sould be among most common'

[('', 804), ('и', 46), ('в', 30), ('иннополис', 21), ('=', 14), ('на', 13), ('по', 13), ('университет', 11), ('университета', 10), ('лаборатория', 10)]


## 4. Crawling ##

Method `crawl_generator()` is given starting url (`source`) and max depth of search. It should return a **generator** of `HtmlDocumentTextData` objects (return a document as soon as it is downloaded and parsed). You can benefit from `yield obj_name` python construction. Use `HtmlDocumentTextData.anchors` field to go deeper.

In [129]:
from queue import Queue

class Crawler:
    
    def crawl_generator(self, source, depth=1):
        q = Queue()
        q.put(source)

        for i in range(depth):
          url = q.get()
          content = HtmlDocumentTextData(url)

          for (text, url) in content.doc.anchors:
            q.put(url)
          
          yield content

### 4.1. Tests ###

In [130]:
crawler = Crawler()
counter = Counter()

for c in crawler.crawl_generator("https://innopolis.university/en/", 2):
    print(c.doc.url)
    if c.doc.url[-4:] in ('.pdf', '.mp3', '.avi', '.mp4', '.txt'):
        print("Skipping", c.doc.url)
        continue
    counter.update(c.get_word_stats())
    print(len(counter), "distinct word(s) so far")
    
print("Done")

print(counter.most_common(20))
assert [x for x in counter.most_common(20) if x[0] == 'innopolis'], 'innopolis sould be among most common'

https://innopolis.university/en/
485 distinct word(s) so far
http://old.innopolis.university/en/
1049 distinct word(s) so far
Done
[('', 1768), ('the', 108), ('of', 89), ('and', 87), ('=', 43), ('innopolis', 40), ('in', 37), ('to', 37), ('for', 37), ('university', 33), ('will', 24), ('research', 24), ('a', 22), ('ул.', 22), ('new', 20), ('it', 20), ('education', 20), ('international', 19), ('programs', 16), ('institute', 16)]
